In [1]:
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel, Phrases
from gensim.models.phrases import Phraser
from gensim.models import LdaModel



# Plotting tools
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
from steam_reviews import ReviewLoader

appid_list = [
211420,
570940,
236430,
335300,
374320,
814380,
1245620,
485510,
1325200,
1448440,
1172380,
1774580,
265300,
678960,
606280,
378540,
644830
]

# language_list = ['english', 'schinese', 'russian']
language_list = ['schinese']
text_data = []

for appid in appid_list:
    for language in language_list:
        file_path_n = 'reviews_' + str(appid) + '_' + language + '_n.json'
        reviews_n = ReviewLoader().load_from_local(file_path_n)
        review_list_n = reviews_n.review_list()
        text_data += review_list_n

len(text_data)

50547

In [3]:
text_data

['中文呢！！！？？？',
 ' 背刺老玩家，现在默认设置没中文了,',
 '繁體中文呢？',
 '下回来，为啥没汉化了？找不着，只有日文英文.',
 '忘不了两次被不死院恶魔打到很久没碰游戏的痛苦\n忘不了坚持玩到火祭场后迫不及待寻找关刀的心情\n忘不了火祭场小隆德飞龙谷夹缝森林刷黑骑的路线\n忘不了掉落关刀后的惊喜和为了能力值反复火祭场旁山坡上刷魂的经历\n第一次遇到大蛞蝓大蠕虫洗衣机的不适感夹杂着对这地图连通性设计的惊叹，让我对这游戏感情复杂\n闭上眼睛，脑海里浮现的便是火祭场，小隆德，飞龙谷，夹缝森林，黑森林庭院，城外不死镇这一大圈的路线与捷径\n第一次经过大树洞到达灰烬湖和第一次通过巨人墓地看到废都和灰烬湖的欣喜也成为了我对这个游戏的珍贵体验\n这个游戏的断尾机制百玩不厌，捏魂bug对我来说也新增了许多独特的体验--在这里我复现了大多数主要的游戏机制bug，包括翻滚捏魂与人性，无敌帧跳病村，大蛇复制物品，法术交换卡入初始火炉\n碎片化的主线支线剧情让我对这个世界充满了探索的欲望，当然了也存在太多地形机关的恶意\n正式开坑两年多的游戏今天终于结束了，黑魂之旅大概还得继续\n2022年11月13日全成就留念\n中评评测，场景实在难以接受甚至引起寻麻疹住院一周，本游戏给差评',
 '最近它把这老版的在线服务器关了',
 '捏马马的游戏本身有质量问题，多年不解决出个重置版你爹我还得又掏了99大洋再好的游戏机制都是给玩家服务的，fs你就是歌姬吧',
 '游戏有问题，平时20帧，病村只有10帧，要玩必须打补丁。后来推出了重置版修复了这个问题，却要玩家再一次掏钱。',
 '为什么不把受死板优化非要出重置版？',
 '什么垃圾服务器，对面玩家都没有攻击到我，就显示掉血',
 '垃圾，爪巴',
 '太黑了 看不清路',
 '牛头恶魔之前不能召唤队友，牛头恶魔的地图设计太差，逼仄的通道，对新手而言是为了为难而为难的设计。第三代的古达明显就比牛头恶魔强很多。\n\n一代明显有很多地方是为了恶心而恶心，这一点很难让人给予好评。',
 '第一个BOSS都过不去的我默默地卸载了游戏并将“困难”标签加入了排除列表。',
 '垃圾游戏',
 '建立角色还要插电视上才能建立否则闪退。\n这些bug这么久了也不修。\n其他移植的也是一堆bug，玩个毛。',
 '人类历史上最狗屎的pc化，这么

In [4]:
import re

# 去除特殊字符和数字
cleaned_text = [re.sub(r'[^\w\s]', '', sentence) for sentence in text_data]
# 转化为小写
lowercase_text = [sentence.lower() for sentence in cleaned_text]

In [5]:
cleaned_text

['中文呢',
 ' 背刺老玩家现在默认设置没中文了',
 '繁體中文呢',
 '下回来为啥没汉化了找不着只有日文英文',
 '忘不了两次被不死院恶魔打到很久没碰游戏的痛苦\n忘不了坚持玩到火祭场后迫不及待寻找关刀的心情\n忘不了火祭场小隆德飞龙谷夹缝森林刷黑骑的路线\n忘不了掉落关刀后的惊喜和为了能力值反复火祭场旁山坡上刷魂的经历\n第一次遇到大蛞蝓大蠕虫洗衣机的不适感夹杂着对这地图连通性设计的惊叹让我对这游戏感情复杂\n闭上眼睛脑海里浮现的便是火祭场小隆德飞龙谷夹缝森林黑森林庭院城外不死镇这一大圈的路线与捷径\n第一次经过大树洞到达灰烬湖和第一次通过巨人墓地看到废都和灰烬湖的欣喜也成为了我对这个游戏的珍贵体验\n这个游戏的断尾机制百玩不厌捏魂bug对我来说也新增了许多独特的体验在这里我复现了大多数主要的游戏机制bug包括翻滚捏魂与人性无敌帧跳病村大蛇复制物品法术交换卡入初始火炉\n碎片化的主线支线剧情让我对这个世界充满了探索的欲望当然了也存在太多地形机关的恶意\n正式开坑两年多的游戏今天终于结束了黑魂之旅大概还得继续\n2022年11月13日全成就留念\n中评评测场景实在难以接受甚至引起寻麻疹住院一周本游戏给差评',
 '最近它把这老版的在线服务器关了',
 '捏马马的游戏本身有质量问题多年不解决出个重置版你爹我还得又掏了99大洋再好的游戏机制都是给玩家服务的fs你就是歌姬吧',
 '游戏有问题平时20帧病村只有10帧要玩必须打补丁后来推出了重置版修复了这个问题却要玩家再一次掏钱',
 '为什么不把受死板优化非要出重置版',
 '什么垃圾服务器对面玩家都没有攻击到我就显示掉血',
 '垃圾爪巴',
 '太黑了 看不清路',
 '牛头恶魔之前不能召唤队友牛头恶魔的地图设计太差逼仄的通道对新手而言是为了为难而为难的设计第三代的古达明显就比牛头恶魔强很多\n\n一代明显有很多地方是为了恶心而恶心这一点很难让人给予好评',
 '第一个BOSS都过不去的我默默地卸载了游戏并将困难标签加入了排除列表',
 '垃圾游戏',
 '建立角色还要插电视上才能建立否则闪退\n这些bug这么久了也不修\n其他移植的也是一堆bug玩个毛',
 '人类历史上最狗屎的pc化这么多bug这么多年一次都没有更新重置一下立刻卖出248的价格业界鬼才运营再好的游戏内容都无法原谅',
 '我讨厌这种买

In [6]:
lowercase_text

['中文呢',
 ' 背刺老玩家现在默认设置没中文了',
 '繁體中文呢',
 '下回来为啥没汉化了找不着只有日文英文',
 '忘不了两次被不死院恶魔打到很久没碰游戏的痛苦\n忘不了坚持玩到火祭场后迫不及待寻找关刀的心情\n忘不了火祭场小隆德飞龙谷夹缝森林刷黑骑的路线\n忘不了掉落关刀后的惊喜和为了能力值反复火祭场旁山坡上刷魂的经历\n第一次遇到大蛞蝓大蠕虫洗衣机的不适感夹杂着对这地图连通性设计的惊叹让我对这游戏感情复杂\n闭上眼睛脑海里浮现的便是火祭场小隆德飞龙谷夹缝森林黑森林庭院城外不死镇这一大圈的路线与捷径\n第一次经过大树洞到达灰烬湖和第一次通过巨人墓地看到废都和灰烬湖的欣喜也成为了我对这个游戏的珍贵体验\n这个游戏的断尾机制百玩不厌捏魂bug对我来说也新增了许多独特的体验在这里我复现了大多数主要的游戏机制bug包括翻滚捏魂与人性无敌帧跳病村大蛇复制物品法术交换卡入初始火炉\n碎片化的主线支线剧情让我对这个世界充满了探索的欲望当然了也存在太多地形机关的恶意\n正式开坑两年多的游戏今天终于结束了黑魂之旅大概还得继续\n2022年11月13日全成就留念\n中评评测场景实在难以接受甚至引起寻麻疹住院一周本游戏给差评',
 '最近它把这老版的在线服务器关了',
 '捏马马的游戏本身有质量问题多年不解决出个重置版你爹我还得又掏了99大洋再好的游戏机制都是给玩家服务的fs你就是歌姬吧',
 '游戏有问题平时20帧病村只有10帧要玩必须打补丁后来推出了重置版修复了这个问题却要玩家再一次掏钱',
 '为什么不把受死板优化非要出重置版',
 '什么垃圾服务器对面玩家都没有攻击到我就显示掉血',
 '垃圾爪巴',
 '太黑了 看不清路',
 '牛头恶魔之前不能召唤队友牛头恶魔的地图设计太差逼仄的通道对新手而言是为了为难而为难的设计第三代的古达明显就比牛头恶魔强很多\n\n一代明显有很多地方是为了恶心而恶心这一点很难让人给予好评',
 '第一个boss都过不去的我默默地卸载了游戏并将困难标签加入了排除列表',
 '垃圾游戏',
 '建立角色还要插电视上才能建立否则闪退\n这些bug这么久了也不修\n其他移植的也是一堆bug玩个毛',
 '人类历史上最狗屎的pc化这么多bug这么多年一次都没有更新重置一下立刻卖出248的价格业界鬼才运营再好的游戏内容都无法原谅',
 '我讨厌这种买

In [7]:
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize

# # 分词
# tokenized_text = [word_tokenize(sentence) for sentence in lowercase_text]

In [8]:
import jieba
tokenized_text = [list(jieba.cut(sentence, cut_all=False)) for sentence in lowercase_text]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YouChain\AppData\Local\Temp\jieba.cache
Loading model cost 0.873 seconds.
Prefix dict has been built successfully.


In [9]:
tokenized_text

[['中文', '呢'],
 [' ', '背刺', '老', '玩家', '现在', '默认设置', '没', '中文', '了'],
 ['繁體', '中文', '呢'],
 ['下', '回来', '为啥', '没', '汉化', '了', '找不着', '只有', '日文', '英文'],
 ['忘不了',
  '两次',
  '被',
  '不',
  '死院',
  '恶魔',
  '打',
  '到',
  '很久没',
  '碰',
  '游戏',
  '的',
  '痛苦',
  '\n',
  '忘不了',
  '坚持',
  '玩',
  '到',
  '火祭',
  '场后',
  '迫不及待',
  '寻找',
  '关刀',
  '的',
  '心情',
  '\n',
  '忘不了',
  '火祭',
  '场',
  '小',
  '隆德',
  '飞龙',
  '谷',
  '夹缝',
  '森林',
  '刷',
  '黑',
  '骑',
  '的',
  '路线',
  '\n',
  '忘不了',
  '掉落',
  '关刀',
  '后',
  '的',
  '惊喜',
  '和',
  '为了',
  '能力',
  '值',
  '反复',
  '火祭',
  '场旁',
  '山坡',
  '上刷魂',
  '的',
  '经历',
  '\n',
  '第一次',
  '遇到',
  '大',
  '蛞蝓',
  '大',
  '蠕虫',
  '洗衣机',
  '的',
  '不适感',
  '夹杂着',
  '对',
  '这',
  '地图',
  '连通性',
  '设计',
  '的',
  '惊叹',
  '让',
  '我',
  '对',
  '这',
  '游戏',
  '感情',
  '复杂',
  '\n',
  '闭上眼睛',
  '脑海',
  '里',
  '浮现',
  '的',
  '便是',
  '火祭',
  '场',
  '小',
  '隆德',
  '飞龙',
  '谷',
  '夹缝',
  '森林',
  '黑森林',
  '庭院',
  '城外',
  '不死',
  '镇',
  '这',
  '一大',
  '圈',
  '的',
  '路线',
  '与',
  '

In [10]:
from nltk.corpus import stopwords
import nltk

# 去除停用词
custom_stopwords = ['黑魂', '游戏', '只狼', '法环', '仁王', '卧龙',
                   '星战', '绝地', '堕落', '代码', '嗜血', '暗黑','血统', '迸发', '\n', ' ']
stop_words = set(stopwords.words('chinese'))
all_stopwords = stop_words.union(custom_stopwords)
filtered_text = [[word for word in sentence if word.lower() not in all_stopwords] for sentence in tokenized_text]
filtered_text

[['中文'],
 ['背刺', '老', '玩家', '默认设置', '没', '中文'],
 ['繁體', '中文'],
 ['下', '回来', '为啥', '没', '汉化', '找不着', '日文', '英文'],
 ['忘不了',
  '两次',
  '不',
  '死院',
  '恶魔',
  '很久没',
  '碰',
  '痛苦',
  '忘不了',
  '玩',
  '火祭',
  '场后',
  '迫不及待',
  '寻找',
  '关刀',
  '心情',
  '忘不了',
  '火祭',
  '场',
  '小',
  '隆德',
  '飞龙',
  '谷',
  '夹缝',
  '森林',
  '刷',
  '黑',
  '骑',
  '路线',
  '忘不了',
  '掉落',
  '关刀',
  '后',
  '惊喜',
  '能力',
  '值',
  '反复',
  '火祭',
  '场旁',
  '山坡',
  '上刷魂',
  '经历',
  '第一次',
  '大',
  '蛞蝓',
  '大',
  '蠕虫',
  '洗衣机',
  '不适感',
  '夹杂着',
  '地图',
  '连通性',
  '设计',
  '惊叹',
  '感情',
  '闭上眼睛',
  '脑海',
  '里',
  '浮现',
  '便是',
  '火祭',
  '场',
  '小',
  '隆德',
  '飞龙',
  '谷',
  '夹缝',
  '森林',
  '黑森林',
  '庭院',
  '城外',
  '不死',
  '镇',
  '一大',
  '圈',
  '路线',
  '捷径',
  '第一次',
  '大树',
  '洞',
  '到达',
  '灰烬',
  '湖',
  '第一次',
  '巨人',
  '墓地',
  '废',
  '都',
  '灰烬',
  '湖',
  '欣喜',
  '珍贵',
  '体验',
  '断尾',
  '机制',
  '百玩',
  '不厌',
  '捏魂',
  'bug',
  '来说',
  '新增',
  '独特',
  '体验',
  '复现',
  '机制',
  'bug',
  '包括',
  '翻滚',
  '捏魂',
  '人性',
  '无敌',
  '

In [11]:
# import nltk
# from nltk.stem import SnowballStemmer

# stemmer = SnowballStemmer("english")

# stemmed_text = [[stemmer.stem(word) for word in sentence] for sentence in filtered_text]

In [12]:
# 准备文本数据和分词
# 这里假设你已经有了一个文本数据集并进行了分词处理

# 创建词典和文档-词频矩阵
dictionary = corpora.Dictionary(filtered_text)  # your_corpus是已分词的文本数据
corpus = [dictionary.doc2bow(text) for text in filtered_text]

# 选择不同主题数进行训练和评估
start = 3  # 主题数的起始值
limit = 10  # 主题数的结束值
step = 1  # 主题数的步长

coherence_scores = []  # 用于存储主题一致性分数
perplexity_scores = []  # 用于存储困惑度分数

for num_topics in range(start, limit, step):
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    
    # 计算主题一致性分数
    coherence_model = CoherenceModel(model=lda_model, texts=filtered_text, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    
    # 计算困惑度分数
    perplexity_score = lda_model.log_perplexity(corpus)
    perplexity_scores.append(perplexity_score)

# 打印结果
print("主题数 vs. 主题一致性分数:")
for num_topics, coherence_score in zip(range(start, limit, step), coherence_scores):
    print(f"{num_topics} 主题: {coherence_score}")

print("\n主题数 vs. 困惑度分数:")
for num_topics, perplexity_score in zip(range(start, limit, step), perplexity_scores):
    print(f"{num_topics} 主题: {perplexity_score}")

主题数 vs. 主题一致性分数:
3 主题: 0.4516739897812605
4 主题: 0.4793673560754653
5 主题: 0.3936291233504822
6 主题: 0.49191250315764745
7 主题: 0.4966063049252692
8 主题: 0.461778943763547
9 主题: 0.4601286747648814

主题数 vs. 困惑度分数:
3 主题: -8.334167563240683
4 主题: -8.34927455557902
5 主题: -8.393870098049646
6 主题: -8.465096949086194
7 主题: -8.562144605421325
8 主题: -8.681063904201725
9 主题: -8.893655233558887


In [11]:
# 创建词袋模型
dictionary = corpora.Dictionary(filtered_text)
# 将文本数据转化为文档-词频矩阵
corpus = [dictionary.doc2bow(text) for text in filtered_text]

In [ ]:
# 训练LDA模型
num_topics = 10
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=42)

In [13]:
# 查看主题
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.200*"垃圾" + 0.065*"视角" + 0.057*"锁定" + 0.035*"狗屎" + 0.017*"狗" + 0.016*"呐" + 0.015*"丢失" + 0.015*"改不了" + 0.015*"打重" + 0.013*"镜头"')
(1, '0.014*"不" + 0.014*"都" + 0.013*"boss" + 0.012*"很" + 0.010*"一个" + 0.009*"地图" + 0.008*"设计" + 0.007*"剧情" + 0.007*"玩家" + 0.007*"说"')
(2, '0.042*"玩" + 0.035*"不" + 0.027*"买" + 0.017*"都" + 0.015*"没" + 0.014*"小时" + 0.012*"还" + 0.012*"存档" + 0.011*"好" + 0.011*"差评"')
(3, '0.075*"优化" + 0.038*"帧" + 0.023*"都" + 0.023*"闪退" + 0.019*"掉" + 0.018*"pc" + 0.016*"不" + 0.013*"玩" + 0.012*"还" + 0.012*"手柄"')
(4, '0.148*"逼" + 0.111*"傻" + 0.105*"妈" + 0.018*"他妈的" + 0.018*"死" + 0.016*"老子" + 0.014*"尼玛" + 0.011*"牛" + 0.011*"操" + 0.008*"玩意"')
(5, '0.065*"联机" + 0.061*"白屏" + 0.042*"橘子" + 0.027*"烂" + 0.022*"服务器" + 0.019*"核显" + 0.016*"进不去" + 0.016*"路痴" + 0.016*"工具" + 0.012*"2k"')
(6, '0.130*"屎" + 0.052*"一坨" + 0.041*"坨" + 0.037*"like" + 0.020*"宫崎" + 0.017*"卡卡" + 0.015*"手感" + 0.015*"英高" + 0.013*"吃" + 0.010*"一卡"')
(7, '0.029*"都" + 0.024*"还" + 0.020*"恶心" + 0.020*"地图" + 0.019*"玩" + 0.018*"人"

In [14]:
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

In [20]:
pyLDAvis.save_html(lda_display, 'output_cn_n.html')

In [21]:
import pickle

# 假设你的LDA模型命名为lda_model
with open('lda_model_cn_t10.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [12]:
import pickle

# 加载LDA模型
with open('lda_model_cn_t10.pkl', 'rb') as f:
    lda_model = pickle.load(f)

In [21]:
topic_distribution = lda_model[corpus[7]]
topic_distribution

[(1, 0.11265671), (3, 0.78118837), (6, 0.06483516)]

In [40]:
from steam_reviews import ReviewLoader

appid_list = [
211420,
570940,
236430,
335300,
374320,
814380,
1245620,
485510,
1325200,
1448440,
1172380,
1774580,
265300,
678960,
606280,
378540,
644830
]

# language_list = ['english', 'schinese', 'russian']
language_list = ['schinese']

topic_mapping = {0: 7, 1: 1, 2: 2, 3: 3, 4: 5, 5: 6, 6: 9, 7: 4, 8: 10, 9: 8}
group_mapping = {2: 0, 10: 0, 3: 1, 6: 2, 1: 3, 4: 4, 5: 5, 7: 5, 8: 5, 9: 5}
index = 0

for appid in appid_list:
    for language in language_list:
        result_list = [0] * 6
        file_path_n = 'reviews_' + str(appid) + '_' + language + '_n.json'
        reviews_n = ReviewLoader().load_from_local(file_path_n)
        review_list_n = reviews_n.review_list()
        for review in review_list_n:
            topics = lda_model[corpus[index]]
            top_topic = max(topics, key=lambda x: x[1])  # 找到概率最高的主题
            topic_index = topic_mapping[top_topic[0]]
            result_list[group_mapping[topic_index]] += 1
            index += 1
            
        print(result_list)
            
        

[37, 49, 1, 42, 8, 11]
[327, 401, 13, 390, 117, 77]
[28, 11, 1, 84, 13, 19]
[226, 136, 18, 1315, 250, 180]
[1539, 455, 178, 1023, 570, 497]
[972, 403, 17, 1199, 734, 680]
[2527, 4335, 384, 4439, 1321, 1160]
[521, 781, 30, 1095, 300, 223]
[368, 831, 38, 786, 298, 212]
[1665, 3989, 52, 2255, 853, 932]
[713, 188, 75, 978, 629, 234]
[231, 523, 9, 146, 62, 82]
[180, 71, 4, 560, 105, 74]
[364, 104, 10, 1316, 416, 190]
[153, 52, 3, 597, 110, 55]
[44, 17, 0, 212, 73, 26]
[88, 102, 4, 207, 84, 40]


#### 